In [2]:
# Initial imports
import pandas as pd
from path import Path

In [4]:
# Load the Housing_data.csv dataset.
file_path = "Resources/Housing_data_Realtor.csv"
housing_df = pd.read_csv(file_path)
print(housing_df.shape)
housing_df.head(10)

(55020, 40)


,month_date_yyyymm,cbsa_code,cbsa_title,HouseholdRank,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,...,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
0,202106,35620,"new york-newark-jersey city, ny-nj-pa",1,617500.0,-0.0120,0.0702,61846,0.0557,-0.0943,...,-0.1670,1.260094e+06,-0.0215,0.1524,94076,0.0785,0.0113,0.521133,0.0322,0.1588
1,202106,31080,"los angeles-long beach-anaheim, ca",2,1024500.0,-0.0234,0.0562,13258,0.0357,-0.2679,...,-0.0334,2.507311e+06,-0.0134,0.1634,28752,0.0118,-0.0743,1.168653,-0.0513,0.4536
2,202106,16980,"chicago-naperville-elgin, il-in-wi",3,354950.0,-0.0001,0.0443,19089,0.1334,-0.3189,...,-0.1228,5.380922e+05,-0.0243,0.0978,44385,0.0422,-0.1180,1.325161,-0.2034,0.5296
3,202106,19100,"dallas-fort worth-arlington, tx",4,387000.0,0.0187,0.0871,6850,0.1591,-0.5903,...,-0.0902,5.952097e+05,0.0118,0.1550,21354,0.0334,-0.3569,2.117372,-0.3789,1.1313
4,202106,26420,"houston-the woodlands-sugar land, tx",5,365995.0,0.0169,0.1261,13440,0.1178,-0.4378,...,-0.0676,5.565350e+05,0.0031,0.1649,30595,0.0356,-0.2320,1.276414,-0.1807,0.6100
5,202106,37980,"philadelphia-camden-wilmington, pa-nj-de-md",6,334950.0,-0.0146,0.0229,9834,0.2053,-0.2102,...,-0.1117,5.044823e+05,-0.0425,0.0598,26086,0.1659,0.6367,1.652634,-0.0896,1.3726
6,202106,47900,"washington-arlington-alexandria, dc-va-md-wv",7,524950.0,0.0295,-0.0091,8860,0.1666,-0.0896,...,-0.1751,7.835916e+05,-0.0044,-0.0114,22613,0.0657,0.5733,1.552257,-0.2414,1.0754
7,202106,33100,"miami-fort lauderdale-west palm beach, fl",8,447000.0,0.0495,0.1189,21316,-0.0508,-0.5164,...,-0.0605,1.193017e+06,0.0420,0.2215,44927,-0.0473,-0.2554,1.107666,0.0076,0.7388
8,202106,12060,"atlanta-sandy springs-roswell, ga",9,395000.0,0.0131,0.1401,15594,0.1033,-0.5273,...,-0.0968,5.506111e+05,-0.0011,0.1831,45421,0.0188,-0.2683,1.912723,-0.2417,1.0311
9,202106,14460,"boston-cambridge-newton, ma-nh",10,699450.0,-0.0006,0.0639,6054,0.1801,-0.2502,...,-0.1430,1.153194e+06,-0.0339,0.1559,8122,0.1314,-0.2369,0.341592,-0.0578,0.0234
